# Editing the SBML Files

I will be using the libSBML python package in order to edit the draft model files that were provided with the Islam paper. This will be used to add reactions that I believe to be missing based upon the information provided in the paper.

Added reactions:
mgk: EX_cpd00035_e0 \
mgk: EX_cpd00107_e0 \
prm: EX_cpd00035_e0

In [27]:
import libsbml

def add_reaction(file_name, metabolite_code, metabolite_name, charge=0):
    # Define the SBML document and model
    reader = libsbml.SBMLReader()
    document = reader.readSBML(f'model files/{file_name}')
    model = document.getModel()

    # Check if the cytosol species already exists
    species_id = f'M_{metabolite_code}_c0'
    if model.getSpecies(species_id) is None:
        species = model.createSpecies()
        species.setId(species_id)
        species.setCompartment('c0')
        species.setName(f'{metabolite_name}[c0]')
        species.setCharge(charge)
        species.setBoundaryCondition(False)

    cytosol_species = model.getSpecies(species_id)

    # create the extracellular species
    species = model.createSpecies()
    species.setId(f'M_{metabolite_code}_e0')
    species.setCompartment('e0')
    species.setName(f'{metabolite_name}[e0]')
    species.setCharge(int(cytosol_species.getCharge()))
    species.setBoundaryCondition(False)


    # Create the export/import reaction
    reaction = model.createReaction()
    reaction.setId(f'EX_{metabolite_code}_e0')
    reaction.setName(f'{metabolite_name} export/import')
    reaction.setReversible(True) # Set reversibility

    # Add reactant to the reaction
    reactant1 = reaction.createReactant()
    reactant1.setSpecies(f'M_{metabolite_code}_e0')  # Use the species ID created above
    reactant1.setStoichiometry(1)
    reactant1.setConstant(False)

    # 6. Define the kinetic law (e.g., Mass Action kinetics: k * S1)
    kinetic_law = reaction.createKineticLaw()
    # Create a parameter for the rate constant
    lb = kinetic_law.createParameter()
    lb.setId('LOWER_BOUND')
    lb.setValue(-1000)
    lb.setName('mmol_per_gDW_per_hr')
    ub = kinetic_law.createParameter()
    ub.setId('UPPER_BOUND')
    ub.setValue(1000)
    ub.setName('mmol_per_gDW_per_hr')
    obj = kinetic_law.createParameter()
    obj.setId('OBJECTIVE_COEFFICIENT')
    obj.setValue(0)
    flx = kinetic_law.createParameter()
    flx.setId('FLUX_VALUE')
    flx.setValue(0)
    flx.setName('mmol_per_gDW_per_hr')


    # Set the math for the kinetic law using MathML
    # You can use libsbml.parseL3Formula to convert from infix string to MathML AST
    math_ml_string = (
        '<math xmlns="http://www.w3.org/1998/Math/MathML">'
        '<ci> FLUX_VALUE </ci>'
        '</math>'
    )
    math_ast = libsbml.readMathMLFromString(math_ml_string)
    if math_ast:
        kinetic_law.setMath(math_ast)
    else:
        print("Error: Could not parse MathML string.")

    # create the reaction to convert from extracellular to cytosol
    transport_reaction = model.createReaction()
    transport_reaction.setId(f'R_{metabolite_code}_transport')
    transport_reaction.setName(f'{metabolite_name} transport')
    transport_reaction.setReversible(True) # Set reversibility

    # Add reactant to the reaction
    cytosol = transport_reaction.createReactant()
    cytosol.setSpecies(f'M_{metabolite_code}_c0')  # Use the species ID created above
    cytosol.setStoichiometry(1)
    cytosol.setConstant(False)

    # Add product to the reaction
    extracellular = transport_reaction.createProduct()
    extracellular.setSpecies(f'M_{metabolite_code}_e0')  # Use the species ID created above
    extracellular.setStoichiometry(1)
    extracellular.setConstant(False)

    # 6. Define the kinetic law (e.g., Mass Action kinetics: k * S1)
    kinetic_law = transport_reaction.createKineticLaw()
    # Create a parameter for the rate constant
    lb = kinetic_law.createParameter()
    lb.setId('LOWER_BOUND')
    lb.setValue(-1000)
    lb.setName('mmol_per_gDW_per_hr')
    ub = kinetic_law.createParameter()
    ub.setId('UPPER_BOUND')
    ub.setValue(1000)
    ub.setName('mmol_per_gDW_per_hr')
    obj = kinetic_law.createParameter()
    obj.setId('OBJECTIVE_COEFFICIENT')
    obj.setValue(0)
    flx = kinetic_law.createParameter()
    flx.setId('FLUX_VALUE')
    flx.setValue(0)
    flx.setName('mmol_per_gDW_per_hr')


    # Set the math for the kinetic law using MathML
    # You can use libsbml.parseL3Formula to convert from infix string to MathML AST
    math_ml_string = (
        '<math xmlns="http://www.w3.org/1998/Math/MathML">'
        '<ci> FLUX_VALUE </ci>'
        '</math>'
    )
    math_ast = libsbml.readMathMLFromString(math_ml_string)
    if math_ast:
        kinetic_law.setMath(math_ast)
    else:
        print("Error: Could not parse MathML string.")

    libsbml.writeSBMLToFile(document, f'model files/{file_name}')

In [ ]:
# add_reaction('M. gottschalkii.xml', 'cpd00035', 'L-Alanine')
# add_reaction('M. gottschalkii.xml', 'cpd00107', 'L-Leucine')
# add_reaction('M. gottschalkii.xml', 'cpd00033', 'Glycine')
# add_reaction('M. gottschalkii.xml', 'cpd00655', 'Dephospho-CoA')
# add_reaction('M. gottschalkii.xml', 'cpd00138', 'D-Mannose')
# add_reaction('M. gottschalkii.xml', 'cpd00082', 'D-Fructose')
# add_reaction('M. gottschalkii.xml', 'cpd01414', 'Tetrathionate', charge=-2)

# add_reaction('P. ruminicola.xml', 'cpd00035', 'L-Alanine')
# add_reaction('P. ruminicola.xml', 'cpd00107', 'L-Leucine')
# add_reaction('P. ruminicola.xml', 'cpd00033', 'Glycine')

# add_reaction('R. flavefaciens.xml', 'cpd00035', 'L-Alanine')
# add_reaction('R. flavefaciens.xml', 'cpd00033', 'Glycine')
